In [1]:
# Createa range of years and use pandas html scraping to create data frame of draft classes by year

In [2]:
import pandas as pd
import time 
from bs4 import BeautifulSoup as bs
import requests


In [3]:
draftYears = range(1999,2021)

ColNames=['DraftYear', 'Rnd', 'Pick', 'Tm', 'Player', 'Pos', 'Age', 'To', 'AP1', 'PB', 'St', 
 'CarAV', 'DrAV', 'G', 'PassingCmp', 'PassingAtt', 'PassingYds', 'PassingTD', 
 'PassingInt', 'RushingAtt', 'RushingYds', 'RushingTD', 'ReceivingRec',
 'ReceivingYds', 'ReceivingTD', 'ReceivingSolo', 'Int', 'Sk', 'College/Univ', 
 'CollegeStats'] 

draftDF = pd.DataFrame(columns = ColNames)


for x in draftYears:
    url = f'https://www.pro-football-reference.com/years/{x}/draft.htm'
    dftLst = pd.read_html(url) # returns list
    tempDF = pd.DataFrame(dftLst[0])
    tempDF.insert(0, 'DraftYear', x)
    tempDF.columns = ColNames
    if x == min(draftYears):
        draftDF = tempDF
    else:
        draftDF = draftDF.append(tempDF, ignore_index=True)
    time.sleep(3)
del tempDF
   
#Drop the College Stats column because it is just the text 'College Stats' - use bs to get the actual href and merge in subsequent steps
draftDF.drop(['CollegeStats'], axis=1, inplace = True)   

In [4]:
#use soup to get the href on each row of the table
# note - there are some players that do not have href for college stats so put in the try except
cStatList = []

for x in draftYears:
    url = f'https://www.pro-football-reference.com/years/{x}/draft.htm'
    
    players = bs(requests.get(f'https://www.pro-football-reference.com/years/{x}/draft.htm').text, 'lxml')\
        .findAll('td', attrs={'data-stat':'player'})
    
    hrefs = bs(requests.get(f'https://www.pro-football-reference.com/years/{x}/draft.htm').text, 'lxml')\
        .findAll('td', attrs={'data-stat':'college_link'})
    
    i = 0
    while True:
        tdict ={}
        try:
            tdict['DraftYear'] = x
            tdict['Player'] = players[i].text
            tdict['CollegeStats'] = hrefs[i].a['href']

        except:
            tdict['DraftYear'] = x
            tdict['Player'] = players[i].text
            tdict['CollegeStats'] = ''

        cStatList.append(tdict)

        i+=1
        if(i > len(players)-1):
            break

In [5]:
#Create a datafrom fome the list that was created
statLinkDF = pd.DataFrame(cStatList)
#StatLinkDF.head()

In [6]:
#Merge the data with the draftDF dataframe (use an inner join- will take care of a flaw in the first df)
draftScrapeDF = pd.merge(left=draftDF, right=statLinkDF, left_on=['DraftYear','Player'], right_on=['DraftYear','Player'])

In [7]:
# commit to sqlite database
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
engine = create_engine('sqlite:///../data/fbdata.sqlite')
draftScrapeDF.to_sql('draftHistory', con=engine, if_exists='replace', index=False)